# Extração de conteúdo de arquivos

## Apache Tika

* Toolkit para detecção de tipo de arquivo e extração de conteúdo e metadados de arquivos;
* Reconhece e extrai conteúdo de mais de 1000 tipos de arquivos;
* Feito em java;
* Possui biblioteca em python que permite uma utilização simples do toolkit;
* Quando devidamente configurado também realiza OCR no conteúdo dos arquivos;
* Utiliza a biblioteca imagemagick (C/C++) para rotacionar e melhorar a qualidade das imagens e a biblioteca Tesseract OCR (C++)


Mais informações em:

* https://imagemagick.org/
* https://github.com/tesseract-ocr/tesseract
* https://tesseract-ocr.github.io/tessdoc/
* https://tika.apache.org/
* https://medium.com/@masreis/text-extraction-and-ocr-with-apache-tika-302464895e5f

## Instalação das bibliotecas

In [37]:
!pip -q install tika
!pip -q install pyarrow

In [38]:
# mostrar o diretório atual (current folder)
%pwd

'/content/drive/My Drive/Colab Notebooks/bootcamp/dados'

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# mudar o diretório ou pasta (folder)
%cd /content/drive/MyDrive/Colab Notebooks/bootcamp/

/content/drive/MyDrive/Colab Notebooks/bootcamp


In [41]:
%cd dados

/content/drive/MyDrive/Colab Notebooks/bootcamp/dados


## Imports e Funções Úteis

In [42]:
from tika import parser
import re
import os
import pandas as pd

def tratar_texto(txt):
    """Função utilitária para remover espaços excessivos e quebras de linhas excessivas de um texto. 
    
    A função ainda remove espaços extras antes de após o texto.
    
    Exemplo de uso:
    
    >>> tratar_texto(' um texto com    alguns espaços extras   ')
    'um texto com alguns espaços extras'
    
    Args:
        txt: Texto para tratamento.
    
    Returns:
        Retorna o texto tratado.    
    """
    
    if not txt:
        txt = ''
        
    if not isinstance(txt, str):
        txt = str(txt)
    
    txt = re.sub(r' [ ]+', r' ', txt)
    txt = re.sub(r'\n\n+', r'\n', txt)
    
    return txt.strip()

def listar_arquivos_pdf():
    """Lista arquivos PDF do diretório atual"""
    arquivos = []
    for root, dirs, files in os.walk('.'):
        for f in files:
            if f.lower().endswith('.pdf') and not f.lower().endswith('-checkpoint.pdf'):
                arquivos.append(f)
    return arquivos
    

## Extração Simples (Sem OCR)

In [43]:
dados = parser.from_file(r'Estatistica Aplicada e Probabilidade para Engenheiros.pdf')
conteudo = tratar_texto(dados['content'])

In [44]:
len(conteudo.split())

726

In [45]:
print(conteudo[:300])

Estatística Aplicada e Probabilidade para Engenheiros, 6ª edição.pdf
Prefácio 
PÚBLICO-ALVO
Este é um livro-texto introdutório para um primeiro curso em estatística aplicada e probabilidade para estudantes de bacharelado
em engenharia e ciências físicas ou químicas. Eles desempenham um papel signifi


## Extração Completa (Com OCR)

Para a extração completa o apache tika depende de uma série de bibliotecas e configurações complicadas.
Isso não é possível fazer dentro do Google Colab.
A forma mais simples de fazer o serviço completo funcionar é utilizando uma imagem docker pré-configurada.

Os comandos abaixo permitem subir o serviço sem qualquer intervenção ou configuração extra:

```
docker pull apache/tika:1.24-full
docker run -it --name tika-server-ocr -d -p 20000:9998 apache/tika:1.24-full

docker exec -it tika-server-ocr /bin/bash
apt-get update
apt-get install tesseract-ocr-por
```


In [46]:
headers = {
    'X-Tika-PDFOcrStrategy': 'ocr_and_text_extraction',
    'X-Tika-OCRLanguage': 'por',
    'X-Tika-PDFextractInlineImages': 'true',
}

dados = parser.from_file(
    'Estatistica Aplicada e Probabilidade para Engenheiros.pdf', 
    serverEndpoint='http://localhost:20000/',
    headers=headers
)

conteudo = tratar_texto(dados['content'])

In [47]:
len(conteudo.split())

726

In [48]:
print(conteudo[:300])

Estatística Aplicada e Probabilidade para Engenheiros, 6ª edição.pdf
Prefácio 
PÚBLICO-ALVO
Este é um livro-texto introdutório para um primeiro curso em estatística aplicada e probabilidade para estudantes de bacharelado
em engenharia e ciências físicas ou químicas. Eles desempenham um papel signifi


# Exercício Rápido (20 min)

Entrar no link abaixo e copiar os arquivos para o diretório bootcamp/dados no seu google drive:

https://drive.google.com/drive/folders/1-Zh-ipKdNlWMa-07ylbOj8q5wUBCStmc?usp=sharing

# Gravar conteúdos textuais grandes ou dataset muito volumosos

O formato parquet é mais indicado para conteúdos textuais grandes. Arquivos CSV não lidam bem com quebras de linhas no conteúdo dos campos. Ainda, é comum que o conteúdo textual tenha o caracter separador do CSV.

Assim, normalmente são utilizados formatos mais avançados que armazenam o conteúdo em binário e permitem compactação dos dados.

Exemplos de formatos:

* parquet: https://parquet.apache.org/
* feather: https://arrow.apache.org/docs/python/feather.html
* HDF5: https://www.hdfgroup.org/solutions/hdf5

In [35]:
# para a utilização de parquet com pandas é necessário garantir apenas que pyarrow ou fastparquet estejam instaladas (apenas uma das libs é suficiente)
